Production 2 

In [1]:
import csv
import json
import pymongo
import re

In [2]:
# Starting database server
client = pymongo.MongoClient('mongodb://localhost:27017/')

In [3]:
# Array for storing names of loaded files
loaded_files = []

In [4]:
# Method converting CSV file to JSON
def csv_to_json(csv_file):
    
    # Extracting file name excluding extension
    file_name = re.split(".csv",csv_file)
    file_name = file_name[0]
    
    # Creating the name for the new JSON file
    json_file = f"{file_name}.json"
    
    # Array to store data from CSV file
    data_array = []
    
    # Extracting data from CSV file
    with open(csv_file, encoding='utf-8') as in_file:
        # Adding data to the array as dictionary objects
        data_reader = csv.DictReader(in_file)
        for row in data_reader:
            data_array.append({file_name:row})
    
    # Writing data to JSON file
    with open(json_file, 'w', encoding='utf-8') as out_file:
        out_file.write(json.dumps(data_array, indent=2))
        # Adding file name to array
        loaded_files.append(json_file)
    

In [5]:
# Method for adding loaded file to Database
def json_to_mongo(json_file):
    
    # Extracting file name excluding extension
    file_name = re.split(".json",json_file)
    file_name = file_name[0]
    
    # Creating a collection with the file name
    collection = db[file_name]
    
    # Adding data from JSON file to collection
    with open(json_file) as db_file:
        data = json.load(db_file)
        for item in data[file_name]:
            ins = collection.insert_one(item)

In [ ]:
while(True):
    
    # Accepting user input
    user_choice = input(
        """Select an option:
        1 - Load a CSV file
        2 - Create a new database
        3 - Add loaded files to database
        4 - Exit\n""")
    
    
    if(user_choice == '1'):
        # Accepting user input and calling method for converting to JSON
        try:
            csv_file = input("Enter the name of your CSV file:")
            csv_to_json(csv_file)
            print(f"{csv} successfully added.")
        # Catching error if file cannot be found
        except FileNotFoundError:
            print("Please enter a valid file name.")
        continue
    
    elif(user_choice == '2'):
        # Accepting user input for database name
        db_name = input("Please enter a name for your database:")
        # Creating database
        db = client[db_name]
        print(f"{db_name} database created.")
        continue
    
    elif(user_choice == '3'):
        try:
            if db_name:
                # Converting all CSV files listed in loaded_file array to JSON
                for file in loaded_files:
                    json_to_mongo(file)
                    print(f"{file} added to {db_name} database.")
        # Catching error if database has not been created
        except NameError:
            print("Please create a database first.")  
        continue
    
    elif(user_choice == '4'):
        print("Exiting program...")
        break
    
    else:
        # Providing advice to the user when an invalid option is entered
        print("Error: Please enter a valid option.")
        continue
    
    # Breaking the While loop (only reached from option 4)
    break